In [56]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from gensim.models import Word2Vec

kernel_sizes =  [1,2,3,4]
class MultiCNNText(): 
    def __init__(self, opt):
        super(MultiCNNText, self).__init__()
        self.model_name = 'MultiCNNTextBNDeep'
        self.opt=opt
        self.encoder = nn.Embedding(opt["vocab_size"],opt["embedding_dim"])
        content_convs = [ nn.Sequential(
                                nn.Conv1d(in_channels = opt["embedding_dim"],
                                        out_channels = opt["content_dim"],
                                        kernel_size = kernel_size),
                                nn.BatchNorm1d(opt["content_dim"]),
                                nn.ReLU(inplace=True),

                                nn.Conv1d(in_channels = opt["content_dim"],
                                        out_channels = opt["content_dim"],
                                        kernel_size = kernel_size),
                                nn.BatchNorm1d(opt["content_dim"]),
                                nn.ReLU(inplace=True),
                                nn.MaxPool1d(kernel_size = (opt["content_seq_len"] - kernel_size*2 + 2))
                            )
            for kernel_size in kernel_sizes ]

        self.content_convs = nn.ModuleList(content_convs)

        self.fc = nn.Sequential(
            nn.Linear(len(kernel_sizes)*(opt.content_dim),opt["linear_hidden_size"]),
            nn.BatchNorm1d(opt["linear_hidden_size"]),
            nn.ReLU(inplace=True),
            nn.sigmoid(opt["linear_hidden_size"])
        )
        

        if opt.embedding_path:
            self.encoder.weight.data.copy_(t.from_numpy(np.load(opt.embedding_path)['vector']))

    def forward(self, content):
        content = self.encoder(content)
        if self.opt.static:
            content.detach()
        content_out = [ content_conv(content.permute(0,2,1)) for content_conv in self.content_convs]
        conv_out = t.cat((content_out),dim=1)
        reshaped = conv_out.view(conv_out.size(0), -1)
        logits = self.fc((reshaped))
        return logits

    # def get_optimizer(self):  
    #    return  t.optim.Adam([
    #             {'params': self.title_conv.parameters()},
    #             {'params': self.content_conv.parameters()},
    #             {'params': self.fc.parameters()},
    #             {'params': self.encoder.parameters(), 'lr': 5e-4}
    #         ], lr=self.opt.lr)
    # # end method forward
    
if __name__ == '__main__':
    opt={}
    opt["content_dim"] = 256
    m = MultiCNNText(opt)
    title = t.autograd.Variable(t.arange(0,500).view(10,50)).long()
    content = t.autograd.Variable(t.arange(0,2500).view(10,250)).long()
    o = m(title,content)
    print(o.size())

KeyError: 'vocab_size'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.fc1 = nn.Linear(768, 32)
        self.fc1 = nn.Linear(768, 1)

    def forward(self, x):
        #x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc1(x))
        return x
    
net = Net()


criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)